In [3]:
pip install psycopg2

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   --------------------------- ------------ 0.8/1.2 MB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 2.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install sqlalchemy

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   -------------- ------------------------- 0.8/2.1 MB 2.6 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.1 MB 2.5 MB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 2.2 MB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 2.2 MB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 2.2 MB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 2.2 MB/s eta 0:00:01
   ----------------------------- ---------- 1.6/2.1 MB 942.3 kB/s eta 0:00:01
   ----------------------------- ---------- 1.6/2.1 MB 942.3 kB/s eta 0:00:01
   ----------------------------- ---------- 1.6/2.1 MB 942.3 kB/s eta 0:00:01
   ----------------------------- ---------- 1.6/2.1 MB 942.3 kB/s eta 0:00:01
   ----------------------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from datetime import datetime
import numpy as np
from IPython.display import display

In [2]:
import pandas as pd
import numpy as np
import psycopg2

file_path = "student_performance_large_dataset.csv"

def Extract():
    SP = pd.read_csv(file_path)
    return SP

def transform(SP):
    # Rename columns for consistency
    SP = SP.rename(columns={
        "Assignment_Completion_Rate (%)": "Assignment_Completion_Rate",
        "Exam_Score (%)": "Exam_Score",
        "Attendance_Rate (%)": "Attendance_Rate"
    })

    # GPA calculation
    def GPA_Scale(Final_Grade):
        if pd.isna(Final_Grade):
            return np.nan
        g = str(Final_Grade).strip().upper()
        if g == "A":
            return 4.0
        elif g == "B":
            return 3.0
        elif g == "C":
            return 2.0
        elif g == "D":
            return 1.0
        else:
            return 0.0

    if "Final_Grade" in SP.columns:
        SP['GPA'] = SP['Final_Grade'].apply(GPA_Scale)

    # Weight assignment & exam scores
    if "Assignment_Completion_Rate" in SP.columns:
        SP['Assignment_Completion_Rate'] = SP['Assignment_Completion_Rate'] * 0.3

    if "Exam_Score" in SP.columns:
        SP['Exam_Score'] = SP['Exam_Score'] * 0.7

    if set(["Assignment_Completion_Rate", "Exam_Score"]).issubset(SP.columns):
        SP['Mastery_Score'] = SP['Assignment_Completion_Rate'] + SP['Exam_Score']

    # Normalize attendance rate
    if "Attendance_Rate" in SP.columns:
        sample = SP["Attendance_Rate"].dropna()
        if len(sample) > 0 and (sample.quantile(0.9) <= 1.5):
            SP["Attendance_Rate"] = SP["Attendance_Rate"] * 100.0

    #  Fix Participation column (convert Yes/No → 1/0)
    if "Participation_in_Discussions" in SP.columns:
        SP["Participation_in_Discussions"] = SP["Participation_in_Discussions"].apply(
            lambda x: 1 if str(x).strip().lower() == "yes" else 0
        )

    return SP

def Load(SP):
    Connection = psycopg2.connect(
        host="localhost",
        dbname="ProjectETL1",
        user="postgres",
        password="postgre123"
    )
    con = Connection.cursor()

    # Ensure a default course exists
    con.execute("""
        INSERT INTO courses(course_id, course_name, term, credits)
        VALUES ('COURSE-101','General Studies','2025 Spring',3)
        ON CONFLICT (course_id) DO NOTHING;
    """)

    # Detect student ID column
    student_id_col = None
    for c in SP.columns:
        if c.lower() in ("student_id", "id", "student"):
            student_id_col = c
            break

    # Insert rows
    for _, row in SP.iterrows():
        sid = str(row[student_id_col]) if student_id_col else None

        # Upsert student
        con.execute("""
            INSERT INTO students(student_id)
            VALUES (%s) ON CONFLICT (student_id) DO NOTHING;
        """, (sid,))

        # Insert performance
        con.execute("""
        INSERT INTO performance(
            student_id, course_id, study_hours_per_week, online_courses_completed, participation_in_discussions,
            assignment_completion_rate, exam_score, mastery_score, final_grade, gpa
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
        """, (
            sid, "COURSE-101",
            row.get("Study_Hours_per_Week"),
            row.get("Online_Courses_Completed"),
            row.get("Participation_in_Discussions"),
            row.get("Assignment_Completion_Rate"),
            row.get("Exam_Score"),
            row.get("Mastery_Score"),
            row.get("Final_Grade"),
            row.get("GPA"),
        ))

        # Insert attendance
        con.execute("""
        INSERT INTO attendance(student_id, course_id, attendance_rate)
        VALUES (%s, %s, %s);
        """, (
            sid, "COURSE-101", row.get("Attendance_Rate")
        ))

    # Commit & close
    Connection.commit()
    con.close()
    Connection.close()

if __name__ == "__main__":
    raw_data = Extract()
    clean_data = transform(raw_data)
    Load(clean_data)
